In [1]:
from re import T
import re
import ast
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import sleep
import os
from difflib import SequenceMatcher
import json
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

import helpers
import api

In [2]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [4]:
df_MCQ = pd.read_csv('results/MCQ_sample_solved_2024_06_04_12_33_53.csv')
df_MCQ_EN = pd.read_csv('results/MCQ_EN_sample_solved_2024_06_04_12_33_53.csv')
df_sample = pd.read_csv('results/provas_enem_EN_sample_solved.csv')
df_sample_EN = df_sample.drop(columns=['CE','DS', 'EK', 'IC', 'MR', 'TC', 'image', 'IU', 'ledor', 'figures'])

from datetime import datetime
date = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
csv_file = f'myfile_{date}.csv'
csv_file
teste = pd.read_csv('all_provas_enem_areas_sys_solved.csv')
teste.to_csv(f'results/teste_{date}.csv', index=False)

In [12]:
df_MCQ['model'].unique()

array(['gpt-3.5-turbo'], dtype=object)

#### Sample 10 questions 

In [20]:
# Padronizar os valores dos níveis de dificuldade para inglês
df_MCQ['difficult_level'] = df_MCQ['difficult_level'].map({
    'Iniciante': 'Beginner',
    'Intermediário': 'Intermediate',
    'Avançado': 'Advanced'
})

# Padronizar os valores dos níveis de Bloom para inglês
df_MCQ['bloom_level'] = df_MCQ['bloom_level'].map({
    'Analisar': 'Analyze',
    'Análise': 'Analyze',
    'Aplicar': 'Apply',
    'Aplicação': 'Apply',
    'Avaliar': 'Evaluate',
    'Avaliação': 'Evaluate',
    'Compreender': 'Understand',
    'Compreensão': 'Understand',
    'Lembrar': 'Remember',
    'Recordar': 'Remember'
})

# Padronizar os valores dos níveis de dificuldade para inglês
df_MCQ_EN['difficult_level'] = df_MCQ['difficult_level'].map({
    'Iniciante': 'Beginner',
    'Intermediário': 'Intermediate',
    'Avançado': 'Advanced'
})

# Padronizar os valores dos níveis de Bloom para inglês
df_MCQ_EN['bloom_level'] = df_MCQ['bloom_level'].map({
    'Analisar': 'Analyze',
    'Análise': 'Analyze',
    'Aplicar': 'Apply',
    'Aplicação': 'Apply',
    'Avaliar': 'Evaluate',
    'Avaliação': 'Evaluate',
    'Compreender': 'Understand',
    'Compreensão': 'Understand',
    'Lembrar': 'Remember',
    'Recordar': 'Remember'
})

df_MCQ['correct_answer'] = df_MCQ['correct_answer'].apply(lambda x: x.replace("'", ""))
df_MCQ['correct_answer'] = df_MCQ['correct_answer'].apply(lambda x: x.replace("[", ""))
df_MCQ['correct_answer'] = df_MCQ['correct_answer'].apply(lambda x: x.replace("]", ""))
df_MCQ_EN['correct_answer'] = df_MCQ_EN['correct_answer'].apply(lambda x: x.replace("'", ""))
df_MCQ_EN['correct_answer'] = df_MCQ_EN['correct_answer'].apply(lambda x: x.replace("[", ""))
df_MCQ_EN['correct_answer'] = df_MCQ_EN['correct_answer'].apply(lambda x: x.replace("]", ""))
df_sample_EN['correct_answer'] = df_sample_EN['correct_answer'].apply(lambda x: x.replace("'", ""))
df_sample_EN['correct_answer'] = df_sample_EN['correct_answer'].apply(lambda x: x.replace("[", ""))
df_sample_EN['correct_answer'] = df_sample_EN['correct_answer'].apply(lambda x: x.replace("]", ""))

In [22]:
def count_equal_and_not_equal(df):
    equal_count = (df['correct_answer'] == df['system_answer']).sum()
    not_equal_count = len(df) - equal_count
    return equal_count, not_equal_count

# DataFrames
data = {
    'df_MCQ': df_MCQ,
    'df_MCQ_EN': df_MCQ_EN
}

# Loop for para aplicar a função a cada DataFrame
for df_name, df in data.items():
    equal_count, not_equal_count = count_equal_and_not_equal(df)
    print(f"Para {df_name}:")
    print("EQUALS:", equal_count)
    print("DISTINCT:", not_equal_count)
    print()


Para df_MCQ:
EQUALS: 98
DISTINCT: 20

Para df_MCQ_EN:
EQUALS: 97
DISTINCT: 21



In [25]:
def count_matches(correct_answers, system_answers, threshold=0.95):
    match_count = 0
    not_match_count = 0

    for i in range(len(correct_answers)):
        correct_answer = correct_answers[i]
        system_answer = system_answers[i]
        ratio = similar(correct_answer, system_answer)
        
        if ratio >= threshold:
            match_count += 1
        else:
            not_match_count += 1

    return match_count, not_match_count


data = {
    'df_MCQ': df_MCQ,
    'df_MCQ_EN': df_MCQ_EN
}


for df_name, df in data.items():
    equal_count, not_equal_count = count_matches(df['correct_answer'], df['system_answer'])
    print(f"DF: {df_name}:")
    print("EQUALS:", equal_count)
    print("DISTINCT:", not_equal_count)
    print()

DF: df_MCQ:
EQUALS: 100
DISTINCT: 18

DF: df_MCQ_EN:
EQUALS: 98
DISTINCT: 20



In [26]:
bloom_level_summary = df_MCQ['bloom_level'].value_counts().sort_index()
print("Bloom Level Summary: ", bloom_level_summary)
difficult_level_summary = df_MCQ['difficult_level'].value_counts().sort_index()
print("Difficult Level Summary: ", difficult_level_summary)
relevance_summary = df_MCQ['relevance'].value_counts().sort_index()
print("Relevance Summary: ", relevance_summary)
adherence_summary = df_MCQ['adherence'].value_counts().sort_index()
print("Adherence Summary: ", adherence_summary)
answerability_summary = df_MCQ['answerability'].value_counts().sort_index()
print("Answerability Summary: ", answerability_summary)
correctness_summary = df_MCQ['correctness'].value_counts().sort_index()
print("Correctness Summary: ", correctness_summary)

Bloom Level Summary:  Series([], Name: count, dtype: int64)
Difficult Level Summary:  Series([], Name: count, dtype: int64)
Relevance Summary:  relevance
4      5
5    113
Name: count, dtype: int64
Adherence Summary:  adherence
3     5
4    81
5    32
Name: count, dtype: int64
Answerability Summary:  answerability
3      1
4      5
5    112
Name: count, dtype: int64
Correctness Summary:  correctness
no       5
yes    113
Name: count, dtype: int64


In [27]:
bloom_level_summary = df_MCQ_EN['bloom_level'].value_counts().sort_index()
print("Bloom Level Summary: ", bloom_level_summary)
difficult_level_summary = df_MCQ_EN['difficult_level'].value_counts().sort_index()
print("Difficult Level Summary: ", difficult_level_summary)
relevance_summary = df_MCQ_EN['relevance'].value_counts().sort_index()
print("Relevance Summary: ", relevance_summary)
adherence_summary = df_MCQ_EN['adherence'].value_counts().sort_index()
print("Adherence Summary: ", adherence_summary)
answerability_summary = df_MCQ_EN['answerability'].value_counts().sort_index()
print("Answerability Summary: ", answerability_summary)
correctness_summary = df_MCQ_EN['correctness'].value_counts().sort_index()
print("Correctness Summary: ", correctness_summary)

Bloom Level Summary:  Series([], Name: count, dtype: int64)
Difficult Level Summary:  Series([], Name: count, dtype: int64)
Relevance Summary:  relevance
5    118
Name: count, dtype: int64
Adherence Summary:  adherence
3      2
4    113
5      3
Name: count, dtype: int64
Answerability Summary:  answerability
4      1
5    117
Name: count, dtype: int64
Correctness Summary:  correctness
0    41
1    77
Name: count, dtype: int64


In [34]:
empty_llama = df_sample_EN['system_answer_llama'].isna().sum()
empty_gpt = df_sample_EN['system_answer_gpt'].isna().sum()
print("Empty Llama: ", empty_llama)
print("Empty GPT: ", empty_gpt)

Empty Llama:  15
Empty GPT:  0


In [35]:
valid_rows = df_sample_EN.dropna(subset=['system_answer_llama', 'system_answer_gpt'])

In [36]:
valid_rows

,id,year,context,question,options,correct_answer,description,area,cor,system_answer,steps_answer_gpt,steps_answer_llama,system_answer_llama,system_answer_gpt
3,46,2017,"\nIf, then, for the things we do there is an e...","\nFor Aristotle, the relationship between the ...",{'A': 'the good of individuals consists in eac...,{C: politics is the science that precedes all ...,NaN,HUMANITIES AND THEIR TECHNOLOGIES,YELLOW,NaN,The text mentions that politics determines whi...,The strategy used to answer this question is t...,{C: politics is the science that precedes all ...,{C: politics is the science that precedes all ...
4,37,2017,\nContranarciso\n\nin me \nI see the other\nan...,\nThe search for identity constitutes a facet ...,"{'A': ' absence of identity traits.\n', 'B': '...",{D: perception of empathy as a factor of self...,NaN,"LANGUAGES, CODES AND THEIR TECHNOLOGIES",YELLOW,NaN,The poem suggests that the 'other' that is in ...,The poem shows the speaker's awareness that th...,{D: perception of empathy as a factor of self-...,{D: perception of empathy as a factor of self-...
5,160,2023,A fisherman has a daily fixed cost of 900 reai...,The minimum number of helpers that this fisher...,"{'A': '1.', 'B': '2.', 'C': '3.', 'D': '4.', '...",{D: 4.},NaN,MATHEMATICS AND ITS TECHNOLOGIES,ORANGE,NaN,To calculate the minimum number of helpers nee...,The fisherman's daily fixed cost is 900 reais ...,{B: 2},{D: 4.}
6,91,2017,\nA common fact when cooking rice is the \nspi...,"\nScientifically, it is known that this color ...",{'A': ' reaction of the cooking gas with the s...,"{B: emission of photons by sodium, excited by...",NaN,NATURAL SCIENCES AND THEIR TECHNOLOGIES,YELLOW,NaN,The color change of the flame occurs due to th...,When the cooking water is spilled onto the blu...,"{B: emission of photons by sodium, excited by ...","{B: emission of photons by sodium, excited by ..."
7,90,2023,Behind the 'magic' of Google Assistant's abili...,The text criticizes technological change due t...,"{'A': 'Function diversification.', 'B': 'Popul...",{E: Professional invisibility.},NaN,HUMANITIES AND THEIR TECHNOLOGIES,ORANGE,NaN,The text criticizes technological change due t...,The passage highlights the issue of profession...,{E: Professional invisibility},{E: Professional invisibility.}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,72,2017,\nBy destroying a landscape of twisted trunk t...,\nThe described process occurs in a biogeograp...,"{'A': 'tropophilic and tropical climate.\n', '...",{A: tropophilic and tropical climate.\n},NaN,HUMANITIES AND THEIR TECHNOLOGIES,YELLOW,NaN,The described process of destroying a landscap...,The passage describes a landscape characterize...,{E: semi-deciduous and humid tropical climate},{B: xerophytic and semi-arid climate.}
96,24,2023,## Slavery\nOlavo Bilac\nThese boys playing sh...,"Published in 1902, Olavo Bilac's text emphasiz...",{'A': 'describe in an impersonal way the conse...,{C: anticipate the future erasure of the marks...,NaN,"LANGUAGES, CODES AND THEIR TECHNOLOGIES",ORANGE,NaN,Olavo Bilac's text emphasizes the evils of sla...,The passage by Olavo Bilac emphasizes the evil...,{B: oppose the privileged childhood of the chi...,{A: describe in an impersonal way the conseque...
97,11,2023,"""I won 25 medals in world championships, seven...",The fact reported in the news draws attention ...,"{'A': 'international athletic performance.', '...",{B: the emotional dimension of athletes.},NaN,"LANGUAGES, CODES AND THEIR TECHNOLOGIES",ORANGE,NaN,The fact reported in the news about Simone Bil...,The passage highlights Simone Biles' experienc...,{B: the emotional dimension of athletes.},{B: the emotional dimension of athletes.}
98,131,2023,The oldest people used to cook beans in an iro...,The strategies mentioned were used with the ai...,"{'A': 'treating diarrhea.', 'B': 'preventing a...",{B: preventing anemia.},NaN,NATURAL SCIENCES AND THEIR TECHNOLOGIES,ORANGE,NaN,"The strategies mentioned, such as coo

In [37]:
equal_values = (valid_rows['system_answer_llama'] == valid_rows['system_answer_gpt']).sum()
different_values = (valid_rows['system_answer_llama'] != valid_rows['system_answer_gpt']).sum()

In [38]:
equal_values

30

In [40]:
different_values

55

In [41]:
len(valid_rows)

85

In [42]:
llama_correct_count = (valid_rows['system_answer_llama'] == valid_rows['correct_answer']).sum()
gpt_correct_count = (valid_rows['system_answer_gpt'] == valid_rows['correct_answer']).sum()

In [43]:
llama_correct_count

20

In [44]:
gpt_correct_count

38

In [67]:
merged_df = pd.merge(df_2013, df_2013_EN, on='id', suffixes=('_2013', '_2013_en'))

common_system_answers = merged_df[merged_df['system_answer_2013'].str[1] == merged_df['system_answer_2013_en'].str[1]]
unique_system_answers = merged_df[merged_df['system_answer_2013'].str[1] != merged_df['system_answer_2013_en'].str[1]]

num_common_ids = len(common_system_answers)
num_unique_ids = len(unique_system_answers)

print("MATCH:", num_common_ids)
print("NOT MATCH:", num_unique_ids)

MATCH: 7
NOT MATCH: 3


In [68]:
merged_df[merged_df['system_answer_2013'].str[1] != merged_df['system_answer_2013_en'].str[1]]

,id,year_2013,context_2013,question_2013,options_2013,correct_answer_2013,description_2013,area_2013,cor_2013,system_answer_2013,...,year_2013_en,context_2013_en,question_2013_en,options_2013_en,correct_answer_2013_en,description_2013_en,area_2013_en,cor_2013_en,system_answer_2013_en,steps_answer_2013_en
3,5,2013,"\nDe ponta a ponta, é tudo praia-palma, ...",\nA carta de Pero Vaz de Caminha permit...,{'A': ' Valorizar a catequese a ser real...,{'A': ' Valorizar a catequese a ser real...,NaN,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,AZUL,{A: Valorizar a catequese a ser realizada sobr...,...,2013,"\nFrom end to end, it's all palm beach, very f...",\nThe letter from Pero Vaz de Caminha allows u...,{'A': ' To value the catechesis to be carried...,{'A': ' To value the catechesis to be carried...,NaN,HUMANITIES AND THEIR TECHNOLOGIES,BLUE,{C: Transmit the knowledge of the indigenous p...,The excerpt from Pero Vaz de Caminha's letter ...
4,6,2013,\nO canto triste dos conquistados: \nos último...,"\nO texto é um registro asteca, cujo se...",{'A': ' tragédia causada pela destruição ...,{'B': ' tentativa frustrada de resistência ...,NaN,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,AZUL,{B: tentativa frustrada de resistência a um po...,...,2013,\nThe sad song of the conquered: \nthe last da...,"\nThe text is an Aztec record, whose meaning i...",{'A': 'the tragedy caused by the destruction o...,{'B': 'the failed attempt to resist a power \n...,NaN,HUMANITIES AND THEIR TECHNOLOGIES,BLUE,{A: the tragedy caused by the destruction of t...,The text describes the devastation and desolat...
9,12,2013,"\nNo final do século XIX, as Grandes So...",\nManifestações culturais como o carnaval ...,{'A': ' distinções sociais eram deixadas de l...,{'D': ' tradições populares se transformavam ...,NaN,CIÊNCIAS HUMANAS E SUAS TECNOLOGIAS,AZUL,{A: distinções sociais eram deixadas de lado e...,...,2013,"\nAt the end of the 19th century, the Great Ca...",\nCultural manifestations such as carnival als...,{'A': 'social distinctions were set aside in t...,{'D': 'popular traditions became a matter of s...,NaN,HUMANITIES AND THEIR TECHNOLOGIES,BLUE,{D: popular traditions became a matter of soci...,The text mentions that the Great Carnival Soci...
